In [ ]:
import PyPDF2

pdf_file = open('Master_D.pdf', 'rb')
pdf_file1 = open('Documento_(1).pdf', 'rb')

dados_pdf = PyPDF2.PdfReader(pdf_file)
numero_paginas = len(dados_pdf.pages)

print(f'O arquivo PDF tem {numero_paginas} páginas.')
for pagina in range(numero_paginas):
    conteudo_pagina = dados_pdf.pages[pagina].extract_text()
    print(f'Conteúdo da página {pagina + 1}:\n{conteudo_pagina}\n')
    
    print(f'Número de palavras na página {pagina + 1}: {len(conteudo_pagina.split())}')
    
    #transformando o arquivo pdf em texto
    with open(f'master_{pagina + 1}.txt', 'w', encoding='utf-8') as arquivo_texto:
        arquivo_texto.write(conteudo_pagina)
        
        
# Unindo dois arquivos PDF
        
merge = PyPDF2.PdfWriter()
    
merge.append(pdf_file)
merge.append(pdf_file1)
merge.write('testando_merged.pdf')
merge.close()


from PyPDF2 import PdfReader, PdfWriter

writer = PdfWriter()
reader = PdfReader('testando_merged.pdf')
for page in range(0, len(reader.pages)):
    
    #lendo cada página do PDF
    
    content_page = reader.pages[page]
     
    mediabox = content_page.mediabox
    
    reader_marc = PdfReader
    
    
    
     
     
     
    
    writer.add_page(reader.pages[page])
    
    
    with open("Master_D.pdf", "wb") as pdf:
        writer.write(pdf)
        

#tranformar imagem em arquivo Pdf

from PIL import Image

image = Image.open("master_img.png")
image.save("master_img.pdf", "PDF", resolution=100.0)

from PyPDF2 import PdfReader

reader = PdfReader("master_img.pdf")
page = reader.pages[0]







    
    
    
   
        
    


O arquivo PDF tem 4 páginas.
Conteúdo da página 1:
Assunto: Nota de agradecimento   
 
Venho por esta via , agradecer a Master D, pelo excelente curso que vem ministrando, 
na qualidade de formando, tenho dado o melhor de mim, para ultrapassar as 
dificuldades e os obstáculos que o curso de Python oferece.  
 
Aos professores, tutores, muito obrigado pelo acompanhamento e, acima de tudo 
pela paciência.  
 
Sou formando em Python, para além disso, sou Desenvolvedor Web, amante de 
tecnologias, que pretende ingressar no mercado de trabalho .  
 
Por ser estrangeiro, tem sido bastante desafiador, estar nesta nobre jornada.  
 
Portanto, acho que, a Master D será capaz de oferecer as bases essências para me 
tornar um profissional na área em Portugal.  
 
Um enorme agradecimento a este nobre centro de formação.  
 
 
De Olgário Catanha.   
Formando número : 10405263  
 
Lisboa, 03 de Junho  de 2025  
 

Número de palavras na página 1: 131
Conteúdo da página 2:
Testando página para Merge  

ModuleNotFoundError: No module named 'reportlab'

In [7]:
#Criação de novo Pdf

from reportlab.pdfgen import canvas

c = canvas.Canvas("Novo.pdf")
c.drawString(100, 750, "Saudações a todo o pessoal da Master D, em especial ao Orientador André!")
c.save()


In [3]:
#PDF adicionar marca d'agua

from PyPDF2 import PdfReader, PdfWriter

reader = PdfReader('master_img.pdf')
image_page = reader.pages[0]
writer = PdfWriter()
reader = PdfReader('Master_D.pdf')

for i in range (0, len(reader.pages)):
    content_page = reader.pages[i]
    
    mediabox = content_page.mediabox
    content_page.merge_page(image_page)
    
    

    
    
    
    writer.add_page(content_page)
    
    
with open('Master_D.pdf', 'wb') as pdf:
    writer.write(pdf)

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

reader = PdfReader('master_img.pdf')
image_page = reader.pages[0]
writer = PdfWriter()
reader = PdfReader('Master_D.pdf')

for i in range (0, len(reader.pages)):
    content_page = reader.pages[i]
    
    mediabox = content_page.mediabox
    content_page.merge_page(image_page)
    
    

    
    
    
    writer.add_page(content_page)
    
    
with open('Master_D.pdf', 'wb') as pdf:
    writer.write(pdf)